In [1]:
import pandas as pd
import numpy as np
import joblib
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Load the CNN model for soil type classification
cnn_model = load_model('cnn_saved_model.h5')

# Load the Random Forest model for crop prediction
crop_model = joblib.load('random_forest_model.joblib')

# Class indices for CNN model's soil type classes (adjust as per your CNN model training)
class_indices = {'alluvial': 0, 'black': 1, 'clay': 2, 'laterite': 3, 'loamy': 4, 'red': 5, 'sandy': 6, 'yellow': 7}  # Replace with your actual class names and indices

# Function to preprocess and predict soil type from image using CNN model
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize image to match model's input size
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale to match training data normalization
    return img_array

def classify_soil_type(cnn_model, img_path, class_indices):
    img_array = preprocess_image(img_path)
    predictions = cnn_model.predict(img_array)[0]  # Get probabilities for each class
    predicted_class_index = np.argmax(predictions)
    predicted_soil_type = {v: k for k, v in class_indices.items()}[predicted_class_index]
    return predicted_soil_type

# Combine predictions for final crop recommendation
def predict_crop(cnn_model, crop_model, img_path, class_indices, nitrogen, phosphorus, potassium):
    # Step 1: Predict soil type from image using CNN
    predicted_soil_type = classify_soil_type(cnn_model, img_path, class_indices)
    print(f"Predicted Soil Type from Image: {predicted_soil_type}")

    # Step 2: Use predefined N, P, K values and the predicted soil type
    input_data = pd.DataFrame([[nitrogen, phosphorus, potassium, predicted_soil_type]],
                              columns=['N', 'P', 'K', 'soil_type'])

    # Step 3: Predict crop using Random Forest model
    predicted_crop = crop_model.predict(input_data)
    
    # Output the recommendation
    print(f"The recommended crop type for the given parameters is: {predicted_crop[0]}")

# Example usage
img_path = '22.jpg'  # Path to the soil image
nitrogen = 37  # Example nitrogen level
phosphorus = 30  # Example phosphorus level
potassium = 34  # Example potassium level

predict_crop(cnn_model, crop_model, img_path, class_indices, nitrogen, phosphorus, potassium)

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step
Predicted Soil Type from Image: yellow


ValueError: Found unknown categories ['yellow'] in column 0 during transform

In [ ]:
multimodal_model.save("multimodal_model.h5")